In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm


%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
train_dir = r'C:\Users\Administrator\OneDrive\Рабочий стол\projects\prostheses\data\ninaprodb1train.pkl'
test_dir = r'C:\Users\Administrator\OneDrive\Рабочий стол\projects\prostheses\data\ninaprodb1test.pkl'

train_data = pd.read_pickle(train_dir)
eval_data = pd.read_pickle(test_dir)

train_data, val_data = train_test_split(train_data, test_size=0.3, random_state=21)

In [13]:
# Кастомный класс Dataset для PyTorch
class Nina1DatasetPyTorch(Dataset):
    def __init__(self, dataframe):
        self.dataframe = dataframe

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        row = self.dataframe.iloc[idx]
        emg_data = row['emg']

        # Обработка данных как в оригинальном классе
        if isinstance(emg_data, list) and len(emg_data) == 1:
            emg_data = emg_data[0]

        data = emg_data[:500]
        if len(data) < 500:
            data = np.concatenate((data, np.zeros((500 - len(data), 10))), axis=0)

        # Изменение формы данных (25, 20, 10)
        input_data = data.reshape((25, 20, 10))
        label = row['stimulus']

        return (
            torch.tensor(input_data, dtype=torch.float32),
            torch.tensor(label, dtype=torch.long)
        )

In [14]:
class EMGHandNetPyTorch(nn.Module):
    def __init__(self, num_classes=52):
        super(EMGHandNetPyTorch, self).__init__()

        # Модифицированная CNN часть
        self.cnn = nn.Sequential(
            nn.Conv1d(10, 64, kernel_size=3, stride=2, padding=1),
            nn.Tanh(),
            nn.MaxPool1d(kernel_size=2, stride=2),

            nn.Conv1d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.Tanh(),
            nn.BatchNorm1d(128),

            nn.Conv1d(128, 256, kernel_size=3, stride=2, padding=1),
            nn.Tanh(),
            nn.BatchNorm1d(256),

            nn.AdaptiveAvgPool1d(4),  # Фиксирует выходной размер
            nn.Flatten()
        )

        # Bi-LSTM часть
        self.bilstm = nn.LSTM(
            input_size=1024,  # 256*4
            hidden_size=200,
            num_layers=1,  # Уменьшаем количество слоев
            bidirectional=True,
            batch_first=True,
            dropout=0.2
        )

        # Полносвязные слои
        self.fc = nn.Sequential(
            nn.Linear(400, 512),  # 200*2 (bidirectional)
            nn.Tanh(),
            nn.Dropout(0.3),
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        batch_size = x.size(0)

        # Обработка временных окон
        x = x.view(-1, 20, 10)  # [batch*25, 20, 10]
        x = x.permute(0, 2, 1)  # [batch*25, 10, 20]

        # CNN обработка
        x = self.cnn(x)  # [batch*25, 256*4=1024]

        # Подготовка к LSTM
        x = x.view(batch_size, 25, -1)  # [batch, 25, 1024]

        # Bi-LSTM
        x, _ = self.bilstm(x)  # [batch, 25, 400]
        x = x[:, -1, :]  # Берем последний временной шаг

        # Классификация
        return self.fc(x)

In [22]:
# Инициализация модели и оптимизатора
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = EMGHandNetPyTorch(num_classes=52).to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-5)
criterion = nn.CrossEntropyLoss()
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

In [23]:
# Создание DataLoader
def create_loaders(train_data, val_data, test_data, batch_size=16):
    train_loader = DataLoader(
        Nina1DatasetPyTorch(train_data),
        batch_size=batch_size,
        shuffle=True
    )
    val_loader = DataLoader(
        Nina1DatasetPyTorch(val_data),
        batch_size=batch_size,
        shuffle=False
    )
    test_loader = DataLoader(
        Nina1DatasetPyTorch(test_data),
        batch_size=batch_size,
        shuffle=False
    )
    return train_loader, val_loader, test_loader

In [ ]:
# Обучение модели
def train_model(model, train_loader, val_loader, epochs=20):
    for epoch in range(epochs):
        model.train()
        train_loss = 0.0
        correct = 0
        total = 0

        for inputs, labels in tqdm(train_loader):
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()

            train_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

        # Валидация
        val_loss = 0.0
        val_correct = 0
        val_total = 0
        model.eval()
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)

                val_loss += loss.item()
                _, predicted = outputs.max(1)
                val_total += labels.size(0)
                val_correct += predicted.eq(labels).sum().item()

        # Обновление learning rate
        scheduler.step()

        # Печать статистики
        print(f'Epoch {epoch+1}/{epochs}')
        print(f'Train Loss: {train_loss/len(train_loader):.4f} | Acc: {100.*correct/total:.2f}%')
        print(f'Val Loss: {val_loss/len(val_loader):.4f} | Acc: {100.*val_correct/val_total:.2f}%')


# Использование:
train_loader, val_loader, test_loader = create_loaders(train_data, val_data, eval_data)
train_model(model, train_loader, val_loader, epochs=20)



100%|██████████| 11/11 [00:00<00:00, 19.43it/s]


Epoch 1/20
Train Loss: 3.9284 | Acc: 2.27%
Val Loss: 3.9464 | Acc: 6.58%


100%|██████████| 11/11 [00:00<00:00, 135.68it/s]


Epoch 2/20
Train Loss: 3.7897 | Acc: 9.66%
Val Loss: 3.9260 | Acc: 2.63%


100%|██████████| 11/11 [00:00<00:00, 130.84it/s]


...

Epoch 18/20
Train Loss: 2.7725 | Acc: 23.86%
Val Loss: 3.0938 | Acc: 14.47%


100%|██████████| 11/11 [00:00<00:00, 117.46it/s]


Epoch 19/20
Train Loss: 2.7664 | Acc: 27.27%
Val Loss: 3.0830 | Acc: 14.47%


100%|██████████| 11/11 [00:00<00:00, 146.67it/s]

Epoch 20/20
Train Loss: 2.7178 | Acc: 30.11%
Val Loss: 3.0758 | Acc: 14.47%
